In [ ]:
######## Implementing Draft 2 of the LR Scheduler ########
######## (Note from Krti: Caleb, feel free to merge whatever parts of this works, ########
######## into the LR Scheduler cell above.)  ########

# If this is not helpful, please feel free to not use!  But let's leave this here for now in case we want to pull anything later.

scheduler = LinearLR(optimizer_ODEnet, start_factor=0.5, total_iters=NUMBER_EPOCHS * len(train_loader))

criterion = torch.nn.MSELoss()
ode_loss_hist = []
ode_test_loss_hist = []
ODEnet_best = None

print('ODENet Training')
for epoch in range(1, NUMBER_EPOCHS + 1):  # Adjusted the range to include the last epoch
    for batch_idx, (inputs, targets) in enumerate(train_loader):
        optimizer_ODEnet.zero_grad()
        outputs = ODEnet(inputs, h=dt, dt=dt, method=method)
        loss = criterion(outputs, targets)
        loss.backward()
        optimizer_ODEnet.step()
        ode_loss_hist.append(loss.item())
        scheduler.step()  # How about: Call the scheduler after each optimization step?

    # Evaluate test loss after each epoch
    for batch_idx, (inputs, targets) in enumerate(test_loader):
        outputs = ODEnet(inputs, h=dt, dt=dt, method=method)
        test_loss = criterion(outputs, targets)
        ode_test_loss_hist.append(test_loss.item())

    if test_loss.item() <= min(ode_test_loss_hist):
        print(f'*** Found new best ODEnet (Epoch: {epoch}, Test Loss: {test_loss.item()}')
        ODEnet_best = copy.deepcopy(ODEnet)

    if epoch % 10 == 0:
        print(f'Epoch: {epoch}, Loss: {loss.item()}')

return ODEnet, ODEnet_best, ode_loss_hist


# In this code:
# I've defined a custom learning rate scheduler LinearLR that decreases the learning rate linearly over the specified number of epochs.
# Inside the training loop, I've added scheduler.step() after each optimization step to update the learning rate.
# I've moved the evaluation of the test loss to be performed after each epoch, so you can track the performance more accurately.
# The best model based on test loss is saved after each epoch if it's better than the previous best.
# The learning rate scheduler is used to adjust the learning rate during training.